In [1]:
from utils import *

# Manipulate data with `ndarray`
<br>
<center><img src="support/ndarray2.gif" width=400></center>

By design similar to numpy

We’ll start by introducing the `NDArray`, MXNet’s primary tool for storing and transforming data. If you’ve worked with `NumPy` before, you’ll notice that a NDArray is, by design, similar to NumPy’s multi-dimensional array. 

## Get started

To get started, let's import the `ndarray` package (`nd` is shortform) from MXNet.

In [2]:
from mxnet import nd

Next, let's see how to create a 2D array (also called a matrix) with values from two sets of numbers: 1, 2, 3 and 4, 5, 6. This might also be referred to as a tuple of a tuple of integers.

NDArray from tuples

In [3]:
nd.array(((1, 2, 3), (5, 6, 7)))


[[1. 2. 3.]
 [5. 6. 7.]]
<NDArray 2x3 @cpu(0)>

We can also create a very simple matrix with the same shape (2 rows by 3 columns) but fill it with 1s.

1s

In [4]:
x = nd.ones(shape=(2, 3))
x


[[1. 1. 1.]
 [1. 1. 1.]]
<NDArray 2x3 @cpu(0)>

Often we’ll want to create arrays whose values are sampled randomly. 
For example, sampling values uniformly between -1 and 1. Here we create the same shape, but with random sampling.

Randomly sampled

In [5]:
y = nd.random.uniform(low=-1, high=1, shape=(2,3))
y


[[0.09762704 0.18568921 0.43037868]
 [0.6885315  0.20552671 0.71589124]]
<NDArray 2x3 @cpu(0)>

As with NumPy, the dimensions of each NDArray are accessible by accessing the `.shape` attribute. We can also query its `size`, which is equal to the product of the components of the shape. In addition, `.dtype` tells the data type of the stored values. dtype is important, support for float16

NDArray metadata

In [6]:
(x.shape, x.size, x.dtype, x.context)

((2, 3), 6, numpy.float32, cpu(0))

Type

In [7]:
nd.ones((2,3), dtype=np.uint8)


[[1 1 1]
 [1 1 1]]
<NDArray 2x3 @cpu(0)>

Context

In [8]:
nd.ones((2,3), ctx=mx.gpu(1))


[[1. 1. 1.]
 [1. 1. 1.]]
<NDArray 2x3 @gpu(1)>

## Operations

NDArray supports a large number of standard mathematical operations. Such as element-wise multiplication:

In [9]:
x * y


[[0.09762704 0.18568921 0.43037868]
 [0.6885315  0.20552671 0.71589124]]
<NDArray 2x3 @cpu(0)>

In [10]:
x.exp()


[[2.7182817 2.7182817 2.7182817]
 [2.7182817 2.7182817 2.7182817]]
<NDArray 2x3 @cpu(0)>

And grab a matrix’s transpose to compute a proper matrix-matrix product:

Matrix multiplication

In [11]:
x.shape, y.shape

((2, 3), (2, 3))

In [12]:
nd.dot(x, y.T)


[[0.71369493 1.6099495 ]
 [0.71369493 1.6099495 ]]
<NDArray 2x2 @cpu(0)>

## Indexing
<br>
<center><img src="support/ndarray.gif" width=400></center>

Read the 2nd and 3d columns

In [13]:
y


[[0.09762704 0.18568921 0.43037868]
 [0.6885315  0.20552671 0.71589124]]
<NDArray 2x3 @cpu(0)>

In [14]:
y[:, 1:3]


[[0.18568921 0.43037868]
 [0.20552671 0.71589124]]
<NDArray 2x2 @cpu(0)>

Writing to a specific element

Writing

In [15]:
y[1:2, 0:2] = 4
y


[[0.09762704 0.18568921 0.43037868]
 [4.         4.         0.71589124]]
<NDArray 2x3 @cpu(0)>

## Converting between MXNet NDArray and NumPy

Converting MXNet NDArrays to and from NumPy is easy. The converted arrays do not share memory.

In [16]:
a = x.asnumpy()
(type(a), a)

(numpy.ndarray, array([[1., 1., 1.],
        [1., 1., 1.]], dtype=float32))

In [17]:
nd.array(a)


[[1. 1. 1.]
 [1. 1. 1.]]
<NDArray 2x3 @cpu(0)>

# Automatic differentiation with `autograd`
<br>
<center><img src="support/autograd.gif" width=200><center>

In [18]:
from mxnet import autograd

differentiate $f(x) = 2 x^2$ with respect to parameter $x$.

In [19]:
x = nd.array([[1, 2], [3, 4]])
x


[[1. 2.]
 [3. 4.]]
<NDArray 2x2 @cpu(0)>

In [20]:
x.attach_grad()

$y=f(x)$

In [21]:
def f(x):
    return 2 * x**2

In [22]:
with autograd.record():
    y = f(x)

In [23]:
x, y

(
 [[1. 2.]
  [3. 4.]]
 <NDArray 2x2 @cpu(0)>, 
 [[ 2.  8.]
  [18. 32.]]
 <NDArray 2x2 @cpu(0)>)

Backpropagation of gradient of y wrt x

In [24]:
y.backward()

$y=2x^2$  

$\frac{dy}{dx} = 4x$

In [25]:
x, x.grad

(
 [[1. 2.]
  [3. 4.]]
 <NDArray 2x2 @cpu(0)>, 
 [[ 4.  8.]
  [12. 16.]]
 <NDArray 2x2 @cpu(0)>)

## Using Python control flows
<br>
<center><img src="support/branching.gif" width=600><center>

$Y=f(X)$
- Take a vector `X` of two random numbers in [-1, 1]
- `X` is multiplied by `2` until its norm reach `1000`
- If `X`'s sum is positive, return 1st element  otherwise 2nd

In [26]:
def f(x):
    x = x * 2
    while x.norm().asscalar() < 1000:
        x = x * 2
    # If sum positive
    # pick 1st
    if x.sum().asscalar() >= 0:
        y = x[0]
    # else pick 2nd
    else:
        y = x[1]
    return y

In [27]:
x = nd.random.uniform(-1, 1, shape=2)
x


[0.08976638 0.6945034 ]
<NDArray 2 @cpu(0)>

In [28]:
x.attach_grad()
with autograd.record():
    y = f(x)
y.backward()

$y=k.x[0]$ or $y=k.x[1]$, hence $\frac{dy}{dx} =  \begin{vmatrix} 0 \\ k \end{vmatrix} $ or $ \begin{vmatrix} k \\ 0 \end{vmatrix}$

with $k = 2^n$ where n is the number of times $x$ was multiplied by 2 

In [29]:
x


[0.08976638 0.6945034 ]
<NDArray 2 @cpu(0)>

In [30]:
x.grad


[2048.    0.]
<NDArray 2 @cpu(0)>